In [20]:
#import stuff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model

In [21]:
#import cleaned up datasets
protein_df = pd.read_csv("data/data_clean/protein_data_clean.csv")
kg_df = pd.read_csv("data/data_clean/kg_data_clean.csv")
kcal_df = pd.read_csv("data/data_clean/kcal_data_clean.csv")
fat_df = pd.read_csv("data/data_clean/fat_data_clean.csv")

<h3>
I want to figure out what the lambda best value to penalize our model is for LASSO, for each dataset set.
I will also print out the weight vector, so we can determine what features LASSO determines are actually important.
</h3>

In [77]:
def loop_lasso(X_train, X_test, y_train, y_test, x):
    best_a = 1
    best_score = 0
    for a in range (1,11):
        Lreg = linear_model.Lasso(alpha = a)
        Lreg.fit(X_train, y_train)
        score = cross_val_score(Lreg, X_test, y_test, cv=5)
        if(np.average(score)>best_score):
            best_score=np.average(score)
            best_a = a  
    print("Best score :",best_score)
    #Get the best model's coefs
    Lreg = linear_model.Lasso(alpha = best_a)
    Lreg.fit(X_train, y_train)
    print("Weight coefficients:")
    for c in range(1,len(Lreg.coef_)):
        if(Lreg.coef_[c]>0.0):
            print(x.columns[c],":",Lreg.coef_[c])
    print('\n')

In [78]:
def LASSO(df):
    x = df.drop(['Country', 'Population','Active','Recovered','Deaths','Confirmed','Undernourished'],axis=1)
    print("Confirmed:")
    #confirmed
    y = df.iloc[:,26].values
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
    loop_lasso(X_train, X_test, y_train, y_test, x)
    print("Deaths:")
    #deaths
    y = df.iloc[:,27].values
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
    loop_lasso(X_train, X_test, y_train, y_test, x)
    print("Recovered:")
    #recovered
    y = df.iloc[:,28].values
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
    loop_lasso(X_train, X_test, y_train, y_test, x)

In [79]:
LASSO(protein_df)

Confirmed:
Best score : 0.3275263394890625
Weight coefficients:
Animal Products_p : 0.049813757170649856
Milk - Excluding Butter_p : 0.09725088024365224
Obesity : 0.06875321896586162


Deaths:
Best score : 0
Weight coefficients:


Recovered:
Best score : 0
Weight coefficients:
Animal Products_p : 0.028958341065788762
Milk - Excluding Butter_p : 0.07166492041246687
Obesity : 0.06008530975303411




In [80]:
LASSO(fat_df)

Confirmed:
Best score : 0.19010576800059031
Weight coefficients:
Animal Products_f : 0.0626877750076944
Obesity : 0.09050490057040118


Deaths:
Best score : 0
Weight coefficients:


Recovered:
Best score : 0.241071722280587
Weight coefficients:
Animal Products_f : 0.032963674992145225
Animal fats_f : 0.002868039477184876
Obesity : 0.07353367053634427




In [81]:
LASSO(kg_df)

Confirmed:
Best score : 0.2582225788481021
Weight coefficients:
Animal Products_kg : 0.048450037807465846
Milk - Excluding Butter_kg : 0.011007300179176855
Obesity : 0.07579652130270749


Deaths:
Best score : 0
Weight coefficients:


Recovered:
Best score : 0
Weight coefficients:
Animal Products_kg : 0.030349927359223584
Milk - Excluding Butter_kg : 0.03320575317494299
Obesity : 0.07165533491604104


